In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from nimosef.data.dataset import NiftiDataset

In [ ]:
data_path="/media/jaume/DATA/Data/Test_NIMOSEF_Dataset"
splits_filename=f"{data_path}/derivatives/manifests_nimosef/dataset_manifest.json"
experiment_name="motion"

mode="train"
results_folder = f"{data_path}/derivatives/nimosef_results"
save_folder_results = os.path.join(results_folder, f"results_{mode}_comparison")
os.makedirs(save_folder_results, exist_ok=True)

In [ ]:
dataset = NiftiDataset(splits_filename, mode=mode)
subjects = dataset.patients
subj_id = subjects[0]

In [ ]:
print(f"Subject: {subj_id}")

pred_boundaries_path = os.path.join(results_folder, subj_id, f'{subj_id}_pred_boundaries.parquet')
pred_boundaries = pd.read_parquet(pred_boundaries_path)

true_boundaries_path = os.path.join(results_folder, subj_id, f'{subj_id}_true_boundaries.parquet')
true_boundaries = pd.read_parquet(true_boundaries_path)

# Flip the z-axis [just for visualization]
pred_boundaries['z'] = -pred_boundaries['z']
true_boundaries['z'] = -true_boundaries['z']

scale_factor = np.array(dataset.bbox) / 2
pred_boundaries['x'] = pred_boundaries['x'] * scale_factor[0]
pred_boundaries['y'] = pred_boundaries['y'] * scale_factor[1]
pred_boundaries['z'] = pred_boundaries['z'] * scale_factor[2]

true_boundaries['x'] = true_boundaries['x'] * scale_factor[0]
true_boundaries['y'] = true_boundaries['y'] * scale_factor[1]
true_boundaries['z'] = true_boundaries['z'] * scale_factor[2]

In [ ]:
frames_ids = np.unique(pred_boundaries['time'])
num_frames = len(frames_ids)

In [ ]:

# Create figure and 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Set axis limits (modify based on your data range for better visualization)
ax.set_xlim([pred_boundaries['x'].min(), pred_boundaries['x'].max()])
ax.set_ylim([pred_boundaries['y'].min(), pred_boundaries['y'].max()])
ax.set_zlim([pred_boundaries['z'].min(), pred_boundaries['z'].max()])

# Create scatter plot (initial frame)
sc = ax.scatter([], [], [])

# Update function for animation
def update(frame):
    ax.clear()
    
    # Reset limits after clearing    
    ax.set_xlim([pred_boundaries['x'].min(), pred_boundaries['x'].max()])
    ax.set_ylim([pred_boundaries['y'].min(), pred_boundaries['y'].max()])
    ax.set_zlim([pred_boundaries['z'].min(), pred_boundaries['z'].max()])
    
    ax.set_title(f"Frame {frame}")  # Show frame number
    plot_pc = pred_boundaries.query(f'time == {frame}')
    ax.scatter(plot_pc['x'], plot_pc['y'], plot_pc['z'])

# Create animation
ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=100)

# Save animation as a .gif
save_animation_path = os.path.join(results_folder, subj_id, f"animation_pred_{experiment_name}.gif")
ani.save(save_animation_path, writer=animation.PillowWriter(fps=30))
plt.close(fig)


In [ ]:

# Create figure and 3D axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Set axis limits (modify based on your data range for better visualization)
ax.set_xlim([true_boundaries['x'].min(), true_boundaries['x'].max()])
ax.set_ylim([true_boundaries['y'].min(), true_boundaries['y'].max()])
ax.set_zlim([true_boundaries['z'].min(), true_boundaries['z'].max()])

# Create scatter plot (initial frame)
sc = ax.scatter([], [], [])

# Update function for animation
def update(frame):
    ax.clear()
    
    # Reset limits after clearing    
    ax.set_xlim([true_boundaries['x'].min(), true_boundaries['x'].max()])
    ax.set_ylim([true_boundaries['y'].min(), true_boundaries['y'].max()])
    ax.set_zlim([true_boundaries['z'].min(), true_boundaries['z'].max()])
    
    ax.set_title(f"Frame {frame}")  # Show frame number
    plot_pc = true_boundaries.query(f'time == {frame}')
    ax.scatter(plot_pc['x'], plot_pc['y'], plot_pc['z'])

# Create animation
ani = animation.FuncAnimation(fig, update, frames=num_frames, interval=100)

# Save animation as a .gif
save_animation_path = os.path.join(results_folder, subj_id, f"animation_true_{experiment_name}.gif")
ani.save(save_animation_path, writer=animation.PillowWriter(fps=30))
plt.close(fig)


In [ ]:
# Assume `pred_boundaries` and `true_boundaries` are DataFrames with columns ['x','y','z','time']
# and `num_frames` is the total number of frames.

fig = plt.figure(figsize=(12, 6))
ax_pred = fig.add_subplot(121, projection='3d')
ax_true = fig.add_subplot(122, projection='3d')

def init():
    # Initialize both axes with proper limits.
    for ax, df in zip([ax_pred, ax_true], [pred_boundaries, true_boundaries]):
        ax.set_xlim([df['x'].min(), df['x'].max()])
        ax.set_ylim([df['y'].min(), df['y'].max()])
        ax.set_zlim([df['z'].min(), df['z'].max()])
    return []

def update(frame):
    # Clear both axes so that old points are removed.
    ax_pred.clear()
    ax_true.clear()
    
    # Reset limits
    for ax, df in zip([ax_pred, ax_true], [pred_boundaries, true_boundaries]):
        ax.set_xlim([df['x'].min(), df['x'].max()])
        ax.set_ylim([df['y'].min(), df['y'].max()])
        ax.set_zlim([df['z'].min(), df['z'].max()])
    
    # Set titles
    ax_pred.set_title(f"Predicted Boundaries - Frame {frame}")
    ax_true.set_title(f"True Boundaries - Frame {frame}")
    
    # Query the DataFrames for the current time step.
    pred_pc = pred_boundaries.query(f'time == {frame}')
    true_pc = true_boundaries.query(f'time == {frame}')
    
    # Plot the point clouds.
    ax_pred.scatter(pred_pc['x'], pred_pc['y'], pred_pc['z'], s=1, c='r')
    ax_true.scatter(true_pc['x'], true_pc['y'], true_pc['z'], s=1, c='b')
    
    return []

ani = animation.FuncAnimation(fig, update, frames=num_frames, init_func=init, interval=100)

save_animation_path = os.path.join(results_folder, subj_id, f"side_by_side_animation_{experiment_name}.gif")
ani.save(save_animation_path, writer=animation.PillowWriter(fps=30))
plt.close(fig)

In [ ]:
path_chamfer = os.path.join(save_folder_results, 'chamfer.parquet')
path_hf95 = os.path.join(save_folder_results, 'hf95.parquet')
path_hf = os.path.join(save_folder_results, 'hf.parquet')

df_chamfer = pd.read_parquet(path_chamfer)
df_hf95 = pd.read_parquet(path_hf95)
df_hf =  pd.read_parquet(path_hf)

In [ ]:
print(df_hf.mean(axis=1).mean(), df_hf.mean(axis=1).std())
print(df_hf95.mean(axis=1).mean(), df_hf95.mean(axis=1).std())
print(df_chamfer.mean(axis=1).mean(), df_chamfer.mean(axis=1).std())

In [ ]:
# Plot them over time
plt.figure()
plt.plot(df_chamfer.loc[subj_id], label='Chamfer distance')
plt.plot(df_hf95.loc[subj_id], label='HF95 distance')
plt.plot(df_hf.loc[subj_id], label='HF distance')
plt.legend()

# Compute the mean and std
mean_chamfer = np.mean(df_chamfer.mean(axis=1))
std_chamfer = np.std(df_chamfer.mean(axis=1))

mean_hf95 = np.mean(df_hf95.mean(axis=1))
std_hf95 = np.std(df_hf95.mean(axis=1))

mean_hf = np.mean(df_hf.mean(axis=1))
std_hf = np.std(df_hf.mean(axis=1))

print(f"Mean chamfer: {mean_chamfer} +- {std_chamfer}")
print(f"Mean hf95: {mean_hf95} +- {std_hf95}")
print(f"Mean hf: {mean_hf} +- {std_hf}")